In [1]:
import pandas as pd
import numpy as np
from time import perf_counter
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from skopt.space import Real, Integer
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)
sig_deg_df = pd.read_csv(f"{dirs.analysis_dir}/{unified_dsets[i]}_sig_DESeq_results_xref_matrisome.tsv", sep = '\t')
matrisome_sig_deg_df = (
    sig_deg_df.query("in_matrisome == True")
        .reset_index(drop=True)
)

# Load and filter survival data

In [5]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["age_at_diagnosis", "bmi", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [6]:
filtered_survival_df = (
    survival_df[["sample_name"] + dep_cols + covariate_cols]
        .query("vital_status == 1")
        .dropna()
        .reset_index(drop=True)
        .pipe(pd.get_dummies, columns=cat_cols)
)
print(filtered_survival_df.shape)
print(filtered_survival_df.shape[0] / survival_df.shape[0])
filtered_survival_df.head()

(48, 13)
0.18532818532818532


,sample_name,vital_status,survival_time,age_at_diagnosis,bmi,race_american indian or alaska native,race_asian,race_black or african american,race_not reported,race_white,ethnicity_hispanic or latino,ethnicity_not hispanic or latino,ethnicity_not reported
0,TCGA-C5-A2LZ-01A-11R-A213-07,1,3046,24059.0,31.992171,0,0,0,0,1,0,1,0
1,TCGA-VS-A9V1-01A-11R-A42T-07,1,157,17001.0,18.730489,0,0,0,0,1,0,0,1
2,TCGA-C5-A1BE-01B-11R-A13Y-07,1,2094,23727.0,34.232692,0,0,0,0,1,0,0,1
3,TCGA-C5-A8XH-01A-11R-A37O-07,1,1394,14444.0,22.582709,0,0,0,0,1,0,1,0
4,TCGA-DS-A7WF-01A-11R-A352-07,1,492,15319.0,24.609375,0,0,0,1,0,1,0,0


# Load normalized matrisome count data

In [7]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_matrisome_survival_counts_t_df = (
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)]
        .set_index("geneID")                        # set as index so will be column names
        .transpose()
        .rename_axis(None, axis=1)                  # column.name will be set to "geneID", we don't want this
        .reset_index()                              # "sample_name" should now be its own column
        .rename({"index": "sample_name"}, axis=1)
)

In [8]:
joined_df = (
    pd.merge(filtered_survival_df, norm_matrisome_survival_counts_t_df, on="sample_name")
        .drop("vital_status", axis=1)
        .set_index("sample_name")
)

# Optimize a SVR

## First, remove columns where std(x) $\approx$ 0

In [9]:
# filtered_joined_df = joined_df.iloc[:, np.where(joined_df.min(axis=0) != joined_df.max(axis=0))[0]]
filtered_joined_df = joined_df.iloc[:, np.where(joined_df.std(axis=0) > 1e-6)[0]]

In [10]:
def objective(h_params, svr_kernel, X, y):
    if svr_kernel == "linear":
        model = SVR(
            kernel=svr_kernel,
            C=h_params[0],
            epsilon=h_params[1]
        )
    elif svr_kernel == "poly":
        model = SVR(
            kernel=svr_kernel,
            C=h_params[0],
            epsilon=h_params[1],
            gamma=h_params[2],
            degree=h_params[3],
            coef0=h_params[4]
        )
    elif svr_kernel == "rbf":
        model = SVR(
            kernel=svr_kernel,
            C=h_params[0],
            epsilon=h_params[1],
            gamma=h_params[2]
        )
    pipeline = make_pipeline(StandardScaler(), model)
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5, shuffle=True),
        n_jobs=-1,
        scoring="neg_mean_absolute_error")
    )

In [11]:
# X = mm_norm_df.iloc[:, 1:].values
# y = mm_norm_df.iloc[:, 0].values
X = filtered_joined_df.iloc[:, 1:].values
y = filtered_joined_df.iloc[:, 0].values

In [12]:
space_lin = [
    Real(1e-1, 1e1, name="C"),
    Real(1e-3, 1e-1, name="epsilon")
]

space_poly = [
    Real(1e-1, 1e1, name="C"),
    Real(1e-3, 1e-1, name="epsilon"),
    Real(1e-3, 1e-1, name="gamma"),
    Integer(1, 3, name="degree"),
    Real(-1, 1, name="coef0")
]

space_rbf = [
    Real(1e-1, 1e1, name="C"),
    Real(1e-3, 1e-1, name="epsilon"),
    Real(1e-3, 1e-1, name="gamma")
]

In [13]:
res_gp_linear = gp_minimize(lambda h_ps: objective(h_ps, "linear", X, y), space_lin, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 5.9135
Function value obtained: 677.1258
Current minimum: 677.1258
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.4523
Function value obtained: 500.4033
Current minimum: 500.4033
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 5.7688
Function value obtained: 749.9280
Current minimum: 500.4033
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 4.5650
Function value obtained: 599.2739
Current minimum: 500.4033
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 4.5373
Function value obtained: 662.2844
Current minimum: 500.4033
Iteration No: 6

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 0.4758
Function value obtained: 545.6280
Current minimum: 500.4033
Iteration No: 19 started. Searching for the next optimal point.


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 0.4575
Function value obtained: 521.5517
Current minimum: 500.4033
Iteration No: 20 started. Searching for the next optimal point.


...

Iteration No: 98 ended. Search finished for the next optimal point.
Time taken: 1.2797
Function value obtained: 582.7127
Current minimum: 499.6628
Iteration No: 99 started. Searching for the next optimal point.


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 1.2628
Function value obtained: 524.4954
Current minimum: 499.6628
Iteration No: 100 started. Searching for the next optimal point.


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 100 ended. Search finished for the next optimal point.
Time taken: 1.4744
Function value obtained: 548.0185
Current minimum: 499.6628


In [14]:
print(f"Best values:\n\tC: {res_gp_linear.x[0]}\n\tepsilon: {res_gp_linear.x[1]}")
res_gp_linear.fun

Best values:
	C: 0.1
	epsilon: 0.1


499.66276943433223

In [15]:
res_gp_poly = gp_minimize(lambda h_ps: objective(h_ps, "poly", X, y), space_poly, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5405
Function value obtained: 533.9901
Current minimum: 533.9901
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 6.1706
Function value obtained: 870.0663
Current minimum: 533.9901
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.2571
Function value obtained: 555.0552
Current minimum: 533.9901
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 4.6328
Function value obtained: 708.3030
Current minimum: 533.9901
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 6.7221
Function value obtained: 961.3792
Current minimum: 533.9901
Iteration No: 6

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 28 ended. Search finished for the next optimal point.
Time taken: 0.5929
Function value obtained: 579.6851
Current minimum: 478.3760
Iteration No: 29 started. Searching for the next optimal point.
Iteration No: 29 ended. Search finished for the next optimal point.
Time taken: 4.6672
Function value obtained: 521.4080
Current minimum: 478.3760
Iteration No: 30 started. Searching for the next optimal point.
Iteration No: 30 ended. Search finished for the next optimal point.
Time taken: 3.0871
Function value obtained: 502.2809
Current minimum: 478.3760
Iteration No: 31 started. Searching for the next optimal point.
Iteration No: 31 ended. Search finished for the next optimal point.
Time taken: 2.6353
Function value obtained: 485.7684
Current minimum: 478.3760
Iteration No: 32 started. Searching for the next optimal point.
Iteration No: 32 ended. Search finished for the next optimal point.
Time taken: 2.1170
Function value obtained: 518.8663
Current minimum: 478.3760
Iteration

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 40 ended. Search finished for the next optimal point.
Time taken: 0.9353
Function value obtained: 544.7846
Current minimum: 478.3760
Iteration No: 41 started. Searching for the next optimal point.


/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 41 ended. Search finished for the next optimal point.
Time taken: 0.9672
Function value obtained: 563.2499
Current minimum: 478.3760
Iteration No: 42 started. Searching for the next optimal point.
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 3.0907
Function value obtained: 517.4691
Current minimum: 478.3760
Iteration No: 43 started. Searching for the next optimal point.
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 2.8663
Function value obtained: 526.9236
Current minimum: 478.3760
Iteration No: 44 started. Searching for the next optimal point.
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 2.5293
Function value obtained: 551.2940
Current minimum: 478.3760
Iteration No: 45 started. Searching for the next optimal point.
Iteration No: 45 ended. Search finished for the next optimal point.
Time taken: 2.6906
Function value obtained: 499.5533
Current minimum: 478.3760
Iteration

In [16]:
print(f"Best values:\n\tC: {res_gp_poly.x[0]}\n\tepsilon: {res_gp_poly.x[1]}\n\tgamma: {res_gp_poly.x[2]}\n\tdegree: {res_gp_poly.x[3]}\n\tcoef0: {res_gp_poly.x[4]}")
res_gp_poly.fun

Best values:
	C: 5.056837849199336
	epsilon: 0.1
	gamma: 0.1
	degree: 1
	coef0: -1.0


478.3760173402753

In [17]:
res_gp_rbf = gp_minimize(lambda h_ps: objective(h_ps, "rbf", X, y), space_rbf, verbose=True)

Iteration No: 1 started. Evaluating function at random point.
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.0130
Function value obtained: 558.0333
Current minimum: 558.0333
Iteration No: 2 started. Evaluating function at random point.
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.0110
Function value obtained: 559.6644
Current minimum: 558.0333
Iteration No: 3 started. Evaluating function at random point.
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 0.0114
Function value obtained: 553.1848
Current minimum: 553.1848
Iteration No: 4 started. Evaluating function at random point.
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 0.0104
Function value obtained: 575.4200
Current minimum: 553.1848
Iteration No: 5 started. Evaluating function at random point.
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 0.0105
Function value obtained: 570.7578
Current minimum: 553.1848
Iteration No: 6

In [18]:
print(f"Best values:\n\tC: {res_gp_rbf.x[0]}\n\tepsilon: {res_gp_rbf.x[1]}\n\tgamma: {res_gp_rbf.x[2]}")
res_gp_rbf.fun

Best values:
	C: 9.907232942279036
	epsilon: 0.027097490950225834
	gamma: 0.07207956201763148


544.6488888888889

In [19]:
svm_opt_results_df = pd.DataFrame.from_dict(
    dict(linear = res_gp_linear.x, poly = res_gp_poly.x, rbf = res_gp_rbf.x),
    orient="index",
    columns=["C", "epsilon", "gamma", "degree", "coef0"]
)
svm_opt_results_df["min"] = [res_gp_linear.fun, res_gp_poly.fun, res_gp_rbf.fun]
svm_opt_results_df = svm_opt_results_df.reset_index().rename(columns={"index": "kernel"})

In [20]:
svm_opt_results_df.to_csv("svr_opt_res.csv", index=False)